# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path="../output_data/cities.csv"
weather_df=pd.read_csv(file_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=weather_df[["Lat", "Lng"]]
humidity=weather_df["Humidity"]
intensity=weather_df["Humidity"].max()
# Plot Heatmap
fig = gmaps.figure(center=(20, 35), zoom_level=2)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=intensity,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_df=weather_df[(weather_df["Max Temp"]<80)&(weather_df["Max Temp"]>70)&(weather_df["Wind"]<10)
                     &(weather_df["Cloudiness"]==0)].dropna().reset_index().drop(columns={"index"})
narrow_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind
0,vallenar,0,CL,1634853168,30,-28.57,-70.76,71.80,7.49
1,ovalle,0,CL,1634853181,38,-30.60,-71.20,70.56,6.60
2,sirsa,0,IN,1634853186,31,29.53,75.02,73.89,5.61
3,mount isa,0,AU,1634853196,53,-20.73,139.50,78.57,0.00
4,duartina,0,BR,1634853204,50,-22.41,-49.40,79.95,7.92
5,beira,0,MZ,1634853219,78,-19.84,34.84,71.69,8.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=narrow_df
hotel_df["Hotel Name"]=""

In [7]:
params={
    "keyword": "hotel",
    "radius":5000,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Beginning Data Retrieval")
print("----------------------------")

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    try:
        print(f"Processing Search in {row['City']} city...")
        response= requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f"{response['results'][0]['name']} has been added to Hotel Names")
    except IndexError:
        print("No Hotel Available")

print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval
----------------------------
Processing Search in vallenar city...
Hotel Origenes has been added to Hotel Names
Processing Search in ovalle city...
Hotel Keo has been added to Hotel Names
Processing Search in sirsa city...
Hotel Gumber has been added to Hotel Names
Processing Search in mount isa city...
Isa Hotel has been added to Hotel Names
Processing Search in duartina city...
HOTEL BOA VISTA has been added to Hotel Names
Processing Search in beira city...
Hotel Tivoli Beira has been added to Hotel Names
-----------------------------
Data Retrieval Complete
-----------------------------


In [8]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind,Hotel Name
0,vallenar,0,CL,1634853168,30,-28.57,-70.76,71.80,7.49,Hotel Origenes
1,ovalle,0,CL,1634853181,38,-30.60,-71.20,70.56,6.60,Hotel Keo
2,sirsa,0,IN,1634853186,31,29.53,75.02,73.89,5.61,Hotel Gumber
3,mount isa,0,AU,1634853196,53,-20.73,139.50,78.57,0.00,Isa Hotel
4,duartina,0,BR,1634853204,50,-22.41,-49.40,79.95,7.92,HOTEL BOA VISTA
5,beira,0,MZ,1634853219,78,-19.84,34.84,71.69,8.01,Hotel Tivoli Beira


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
heat_layer = gmaps.heatmap_layer(locations)
marker_layer= gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))